<a href="https://colab.research.google.com/github/jinyang628/textSummarisationNLP/blob/main/textSummarisation.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install transformers[sentencepiece] datasets sacrebleu rouge_score py7zr -q
!pip install --upgrade accelerate

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.1/7.1 MB 43.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 474.6/474.6 kB 20.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 118.9/118.9 kB 13.6 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 66.4/66.4 kB 3.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 236.8/236.8 kB 25.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.8/7.8 MB 118.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 84.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.1/1.1 MB 74.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 110.5/110.5 kB 16.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 212.5/212.5 kB 29.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.3/134.3 kB 20.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 77.9 MB/s eta 0:00:

In [ ]:
!nvidia-smi

Tue Jun  6 08:23:33 2023       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 525.85.12    Driver Version: 525.85.12    CUDA Version: 12.0     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla T4            Off  | 00000000:00:04.0 Off |                    0 |
| N/A   39C    P8     9W /  70W |      0MiB / 15360MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [ ]:
"""
transformers is a library developed by Hugging Face that provides pre-trained 
models and various utilities for natural language processing (NLP) tasks
"""
from transformers import pipeline, set_seed
from transformers import AutoModelForSeq2SeqLM, AutoTokenizer
from transformers import DataCollatorForSeq2Seq
from transformers import TrainingArguments, Trainer


import matplotlib.pyplot as plt
from datasets import load_dataset
import pandas as pd
from datasets import load_dataset, load_metric

import nltk

# tokenize paragraph into sentences
from nltk.tokenize import sent_tokenize

from tqdm import tqdm
import torch

# punkt tokenizer is used for sentence tokenization
nltk.download("punkt")

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


True

In [ ]:
# Select the GPU if GPU is available
device = "cuda" if torch.cuda.is_available() else "cpu"

In [ ]:
# transformer-based model trained on the CNN/Daily Mail dataset for text summarization.
model_ckpt = "google/pegasus-cnn_dailymail"

# AutoTokenizer class is used to load the appropriate tokenizer for a specific pre-trained model
tokenizer = AutoTokenizer.from_pretrained(model_ckpt)

In [ ]:
"""
AutoModelForSeq2SeqLM class is used to load the appropriate pre-trained 
sequence-to-sequence model for a specific task, in this case, language modeling

Sequence to Sequence because we are taking input text and generating output text
"""
model_pegasus = AutoModelForSeq2SeqLM.from_pretrained(model_ckpt).to(device)

In [ ]:
# The dataset we have is the id, a dialogue between two people, the summary
dataset_samsum = load_dataset("samsum")

Generating train split:   0%|          | 0/14732 [00:00<?, ? examples/s]

Generating test split:   0%|          | 0/819 [00:00<?, ? examples/s]

Generating validation split:   0%|          | 0/818 [00:00<?, ? examples/s]

Dataset samsum downloaded and prepared to /root/.cache/huggingface/datasets/samsum/samsum/0.0.0/f1d7c6b7353e6de335d444e424dc002ef70d1277109031327bc9cc6af5d3d46e. Subsequent calls will reuse this data.


  0%|          | 0/3 [00:00<?, ?it/s]

In [ ]:
# calculates the lengths of different splits in the dataset
# 14732 training, 819 testing, 818 validation
split_lengths = [len(dataset_samsum[split]) for split in dataset_samsum]

In [ ]:
"""
creates a pipeline object for text summarization using the specified 
pre-trained model checkpoint
"""
pipe = pipeline('summarization', model = model_ckpt )

In [ ]:
def generate_batch_sized_chunks(list_of_elements, batch_size):
  # Splits the dataset into smaller batches
  # Prevents the programme from loading the entire dataset into memory at once 
  for i in range(0, len(list_of_elements), batch_size):
    yield list_of_elements[i : i + batch_size]

"""
Note that this metric is specifically designed for text summarisation tasks
using the PEGASUS model. It is NOT generalisable to any seq2seq problems
"""
def calculate_metric_on_test_ds(dataset, metric, model, tokenizer, 
                               batch_size=16, device=device, 
                               column_text="article", 
                               column_summary="highlights"):
  # divides the input dataset into smaller batches
  article_batches = list(generate_batch_sized_chunks(dataset[column_text], batch_size))
  target_batches = list(generate_batch_sized_chunks(dataset[column_summary], batch_size))

  for article_batch, target_batch in tqdm(
      zip(article_batches, target_batches), total=len(article_batches)):
      
      # tokenizes the article batch using the tokenizer  
      inputs = tokenizer(article_batch, max_length=1024,  truncation=True, 
                      padding="max_length", return_tensors="pt")
      
      """
      Generate method applies the pre-trained model to generate summaries
      
      input_ids: Represents the input sequence converted into token IDs. It 
      contains the tokenized representation of the input text.

      attention_mask: Binary mask that indicates which tokens should be attended 
      to and which should be ignored. It is used to specify which tokens are 
      padding tokens (0) and which are actual tokens (1). It helps the model 
      focus on the relevant parts of the input.

      length_penalty: Controls the penalty for generating longer sequences. A 
      value less than 1.0 encourages the model to generate shorter summaries, 
      while a value greater than 1.0 encourages longer summaries. In this case, 
      a length penalty of 0.8 is used, indicating a slight preference for 
      shorter summaries.

      num_beams: This parameter controls the number of beams to use during beam 
      search decoding. Beam search is an algorithm used to generate multiple 
      possible sequences in parallel and select the most likely sequence as the 
      output. Setting num_beams to 8 means that the model generates 8 different 
      sequences and selects the one with the highest probability.

      max_length: This parameter sets the maximum length (in terms of tokens) 
      for the generated summaries. If a generated summary exceeds this length, 
      it will be truncated to the specified maximum length.
      """
      summaries = model.generate(input_ids=inputs["input_ids"].to(device),
                        attention_mask=inputs["attention_mask"].to(device), 
                        length_penalty=0.8, num_beams=8, max_length=128)

      # Decode the generated texts
      decoded_summaries = [tokenizer.decode(s, skip_special_tokens=True, 
                              clean_up_tokenization_spaces=True) 
              for s in summaries]      
      decoded_summaries = [d.replace("", " ") for d in decoded_summaries]
      
      # Record the batch's performance (generated summary vs given summary) 
      metric.add_batch(predictions=decoded_summaries, references=target_batch)
      
  # Compute and return the score with reference to the parameter metric.
  score = metric.compute()
  return score

In [ ]:
# ROUGE measures similarity between the generated and given summaries
rouge_metric = load_metric('rouge')

score = calculate_metric_on_test_ds(dataset_samsum['test'], rouge_metric, model_pegasus, tokenizer, column_text = 'dialogue', column_summary = 'summary', batch_size = 8)

<ipython-input-11-182a6c1b8cab>:2: FutureWarning: load_metric is deprecated and will be removed in the next major version of datasets. Use 'evaluate.load' instead, from the new library 🤗 Evaluate: https://huggingface.co/docs/evaluate
  rouge_metric = load_metric('rouge')


100%|██████████| 103/103 [17:50<00:00, 10.39s/it]


In [ ]:
# Check performance on base pegasus model 
rouge_names = ["rouge1", "rouge2", "rougeL", "rougeLsum"]
rouge_dict = dict((rn, score[rn].mid.fmeasure ) for rn in rouge_names )

pd.DataFrame(rouge_dict, index = ['pegasus'])

,rouge1,rouge2,rougeL,rougeLsum
pegasus,0.01554,0.000298,0.015501,0.015534


In [ ]:
"""
Tokenizes and encodes the input dialogue and target summary, truncates them to 
specified maximum lengths, and prepares the dataset for training the 
summarization model by creating input encodings, attention masks, and labels.

Responsible for converting individual examples into the desired format
"""
def convert_examples_to_features(example_batch):
    input_encodings = tokenizer(example_batch['dialogue'] , max_length = 1024, truncation = True )
    
    with tokenizer.as_target_tokenizer():
        target_encodings = tokenizer(example_batch['summary'], max_length = 128, truncation = True )
        
    return {
        'input_ids' : input_encodings['input_ids'],
        'attention_mask': input_encodings['attention_mask'],
        'labels': target_encodings['input_ids']
    }
    
dataset_samsum_pt = dataset_samsum.map(convert_examples_to_features, batched = True)

"""
Handles the collation process for the entire dataset, taking care of padding, 
truncation, attention masks, and label preparation
"""
seq2seq_data_collator = DataCollatorForSeq2Seq(tokenizer, model=model_pegasus)

Map:   0%|          | 0/14732 [00:00<?, ? examples/s]

/usr/local/lib/python3.10/dist-packages/transformers/tokenization_utils_base.py:3606: UserWarning: `as_target_tokenizer` is deprecated and will be removed in v5 of Transformers. You can tokenize your labels by using the argument `text_target` of the regular `__call__` method (either in the same call as your input texts if you use the same keyword arguments, or in a separate call.
  warnings.warn(


Map:   0%|          | 0/819 [00:00<?, ? examples/s]

Map:   0%|          | 0/818 [00:00<?, ? examples/s]

In [ ]:
"""
warmup_steps: The number of steps during which the learning rate will be 
gradually increased from 0 to its maximum value. Warmup steps help stabilize 
training and prevent the model from getting stuck in suboptimal solutions early on.

weight_decay: The amount of weight decay applied to the model's parameters 
during optimization. Weight decay helps prevent overfitting by adding a penalty 
to the loss function based on the magnitude of the model's weights.

evaluation_strategy: Determines when to evaluate the model on the validation 
dataset. Possible values include 'no' (no evaluation), 'steps' (evaluate at the 
specified eval_steps), or 'epoch' (evaluate at the end of each epoch).

gradient_accumulation_steps: The number of gradient accumulation steps before 
performing a backward pass and updating the model's parameters. This parameter 
allows for larger effective batch sizes by accumulating gradients over multiple steps.
"""

trainer_args = TrainingArguments(
    output_dir='pegasus-samsum', num_train_epochs=1, warmup_steps=500,
    per_device_train_batch_size=1, per_device_eval_batch_size=1,
    weight_decay=0.01, logging_steps=10,
    evaluation_strategy='steps', eval_steps=500, save_steps=1e6,
    gradient_accumulation_steps=16
) 

In [ ]:
trainer = Trainer(model=model_pegasus, args=trainer_args,
                  tokenizer=tokenizer, data_collator=seq2seq_data_collator,
                  train_dataset=dataset_samsum_pt["train"], 
                  eval_dataset=dataset_samsum_pt["validation"])

In [ ]:
trainer.train()

/usr/local/lib/python3.10/dist-packages/transformers/optimization.py:407: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(
You're using a PegasusTokenizerFast tokenizer. Please note that with a fast tokenizer, using the `__call__` method is faster than using a method to encode the text followed by a call to the `pad` method to get a padded encoding.


Step,Training Loss,Validation Loss
500,1.690900,1.484756


TrainOutput(global_step=920, training_loss=1.8256099980810414, metrics={'train_runtime': 2716.4901, 'train_samples_per_second': 5.423, 'train_steps_per_second': 0.339, 'total_flos': 5526961323663360.0, 'train_loss': 1.8256099980810414, 'epoch': 1.0})

In [ ]:
# Check performance after training
score = calculate_metric_on_test_ds(
    dataset_samsum['test'], rouge_metric, trainer.model, tokenizer, batch_size = 2, column_text = 'dialogue', column_summary= 'summary'
)

rouge_dict = dict((rn, score[rn].mid.fmeasure ) for rn in rouge_names )

pd.DataFrame(rouge_dict, index = [f'pegasus'] )

100%|██████████| 410/410 [13:09<00:00,  1.93s/it]


,rouge1,rouge2,rougeL,rougeLsum
pegasus,0.018561,0.000327,0.018426,0.018432


In [ ]:
# Save model
model_pegasus.save_pretrained("pegasus-samsum-model")

In [ ]:
# Save tokenizer
tokenizer.save_pretrained("tokenizer")

('tokenizer/tokenizer_config.json',
 'tokenizer/special_tokens_map.json',
 'tokenizer/spiece.model',
 'tokenizer/added_tokens.json',
 'tokenizer/tokenizer.json')

In [ ]:
# Use model on custom inputs to generate summaries
tokenizer = AutoTokenizer.from_pretrained("tokenizer")
dataset_samsum = load_dataset("samsum")
# Can write our own input here 
sample_text = dataset_samsum["test"][0]["dialogue"]
gen_kwargs = {"length_penalty": 0.8, "num_beams": 8, "max_length": 128}
pipe = pipeline("summarization", model="pegasus-samsum-model", tokenizer=tokenizer)

# original text + generated summary 
print(sample_text)
print(pipe(sample_text, **gen_kwargs)[0]["summary_text"])

  0%|          | 0/3 [00:00<?, ?it/s]

Your max_length is set to 128, but your input_length is only 122. Since this is a summarization task, where outputs shorter than the input are typically wanted, you might consider decreasing max_length manually, e.g. summarizer('...', max_length=61)


Hannah: Hey, do you have Betty's number?
Amanda: Lemme check
Hannah: <file_gif>
Amanda: Sorry, can't find it.
Amanda: Ask Larry
Amanda: He called her last time we were at the park together
Hannah: I don't know him well
Hannah: <file_gif>
Amanda: Don't be shy, he's very nice
Hannah: If you say so..
Hannah: I'd rather you texted him
Amanda: Just text him 🙂
Hannah: Urgh.. Alright
Hannah: Bye
Amanda: Bye bye
Amanda can't find Betty's number. Larry called Betty's last time they were at the park together. Hannah wants Amanda to text him.
